# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [14]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [15]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
# for root, dirs, files in os.walk(filepath):
    
file_path_list = glob.glob(os.path.join(filepath,'*'))

# print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [16]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
#             print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [17]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [18]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])

    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [19]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [20]:
# Set keysapce 
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## The collection fo Data for creating music_seesion_history table

In [21]:
df=pd.read_csv('event_datafile_new.csv')
df=df[['itemInSession','sessionId','artist','song','length']]
df.head()

,itemInSession,sessionId,artist,song,length
0,0,961,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Horn Concerto No. 4 in E flat K495: II. Romanc...,277.15873
1,1,961,Jimi Hendrix,Woodstock Inprovisation,239.82975
2,2,961,Building 429,Majesty (LP Version),300.61669
3,0,107,The B-52's,Love Shack,321.54077
4,1,107,Die Mooskirchner,Frisch und g'sund,169.29914


In [28]:
# check the data
df[(df['sessionId']==338)&(df['itemInSession']==4)]

,itemInSession,sessionId,artist,song,length
444,4,338,Faithless,Music Matters (Mark Knight Dub),495.3073


## Query 1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [23]:
# Create table by query statement
query = "CREATE TABLE IF NOT EXISTS music_seesion_history"
query = query + "(session_id int, item_in_session int, artist text, song text, length float, PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute('DROP TABLE IF EXISTS music_seesion_history')
    session.execute(query)
    
except Exception as e:
    print(e)
                    

### Load data and Insert sparkify's data into table by column's name

In [24]:
# read from csv file line by line
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO music_seesion_history (session_id, item_in_session, artist, song, length)" 
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        # Insert sessionId itemInSession artist, song, length to table 
        session.execute(query, (int(line[8]),int(line[3]),str(line[0]),str(line[9]),float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [27]:
session_id = 338
item_in_session = 4

query = f'SELECT artist, song, length FROM music_seesion_history WHERE session_id={session_id} and item_in_session={item_in_session}'

try:
    rows = session.execute(query)
    
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [31]:
# check the data
df=pd.read_csv('event_datafile_new.csv')
df=df[['userId','sessionId','artist','song','itemInSession','firstName','lastName']]
df.head()

,userId,sessionId,artist,song,itemInSession,firstName,lastName
0,88,961,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Horn Concerto No. 4 in E flat K495: II. Romanc...,0,Mohammad,Rodriguez
1,88,961,Jimi Hendrix,Woodstock Inprovisation,1,Mohammad,Rodriguez
2,88,961,Building 429,Majesty (LP Version),2,Mohammad,Rodriguez
3,38,107,The B-52's,Love Shack,0,Gianna,Jones
4,38,107,Die Mooskirchner,Frisch und g'sund,1,Gianna,Jones


In [48]:
df[(df['userId']==10)&(df['sessionId']==182)].sort_values(by='itemInSession')

,userId,sessionId,artist,song,itemInSession,firstName,lastName
4704,10,182,Down To The Bone,Keep On Keepin' On,0,Sylvie,Cruz
4705,10,182,Three Drives,Greece 2000,1,Sylvie,Cruz
4706,10,182,Sebastien Tellier,Kilometer,2,Sylvie,Cruz
4707,10,182,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,3,Sylvie,Cruz


### Create table by query statement

In [49]:
query = "CREATE TABLE IF NOT EXISTS music_user_history"
query = query + "(user_id int, session_id int, item_in_session int, artist text, song text, first_name text, last_name text, PRIMARY KEY ((user_id, session_id), item_in_session))"

try:
    session.execute('DROP TABLE IF EXISTS music_user_history')
    session.execute(query)
    
except Exception as e:
    print(e)

### Load data and Insert sparkify's data into table by column's name

In [50]:
# read from csv file line by line
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO music_user_history (user_id, session_id, item_in_session, artist, song, first_name, last_name)" 
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Insert values 
        session.execute(query, (int(line[-1]),int(line[8]),int(line[3]),str(line[0]),str(line[9]),str(line[1]),str(line[4])))

### Do a SELECT to verify that the data have been inserted into each table

In [54]:
user_id = 10
session_id = 182
# query artist, song, first name, last name by user id and seesion id order by item in session ASC or DESC
query = f'SELECT artist, song, first_name, last_name FROM music_user_history WHERE  user_id={user_id} and session_id={session_id} ORDER BY item_in_session ASC'    

try:
    rows = session.execute(query)
    
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


## Query 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [59]:
#check the data
df=pd.read_csv('event_datafile_new.csv')
df=df[['song','userId','firstName','lastName']]
df.head()

,song,userId,firstName,lastName
0,Horn Concerto No. 4 in E flat K495: II. Romanc...,88,Mohammad,Rodriguez
1,Woodstock Inprovisation,88,Mohammad,Rodriguez
2,Majesty (LP Version),88,Mohammad,Rodriguez
3,Love Shack,38,Gianna,Jones
4,Frisch und g'sund,38,Gianna,Jones


In [75]:
df[df['song']=='All Hands Against His Own'].sort_values(by='userId')

,song,userId,firstName,lastName
6298,All Hands Against His Own,29,Jacqueline,Lynch
2792,All Hands Against His Own,80,Tegan,Levine
5135,All Hands Against His Own,95,Sara,Johnson


### Create table by query statement

In [65]:
query = "CREATE TABLE IF NOT EXISTS music_song_history"
query = query + "(song text, user_id int, first_name text, last_name text, PRIMARY KEY (song, user_id))"

try:
    session.execute('DROP TABLE IF EXISTS music_song_history')
    session.execute(query)
    
except Exception as e:
    print(e)

### Load data and Insert sparkify's data into table by column's name

In [67]:
# read from csv file line by line
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO music_song_history (song, user_id, first_name, last_name)" 
        query = query + "VALUES (%s, %s, %s, %s)"
        # Insert values 
        session.execute(query, (str(line[9]),int(line[-1]),str(line[1]),str(line[4])))

### Do a SELECT to verify that the data have been inserted into each table

In [74]:
query = "SELECT first_name, last_name FROM music_song_history WHERE  song='All Hands Against His Own' ORDER BY user_id ASC"

try:
    rows = session.execute(query)
    
except Exception as e:
    print(e)

for row in rows:
    print(f'{row.first_name} {row.last_name}')

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [ ]:
try:
    session.execute('DROP TABLE IF EXISTS music_user_history')
    session.execute('DROP TABLE IF EXISTS music_user_history')
    session.execute('DROP TABLE IF EXISTS music_song_history')
    
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()